In [1]:
!pip install transformers
!pip install torch
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
df = pd.read_csv("/content/Test_file.txt", engine='python',encoding='utf-8', error_bad_lines=False)
df.columns = ["cat", "first", "second", "third","fourth"]
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,cat,first,second,third,fourth
0,city,اهواز,خوزستان,قشم,هرمزگان
1,city,شیراز,فارس,دامغان,سمنان
2,city,رشت,گیلان,اسلام‌شهر,تهران
3,city,بندرعباس,هرمزگان,ملارد,تهران
4,city,اراک,مرکزی,مریوان,کردستان


In [3]:
tokenizer_ParsBERT = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')
model_ParsBERT = BertForMaskedLM.from_pretrained('HooshvareLab/bert-fa-base-uncased')

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def predict_masked_word(sentence, model="ParsBERT",relationship=0):
    """ Predicts the masked word in a given sentence using a transformer model and it's tokenizer. """
    # Model Selection
    if(model=="ParsBERT"):
      model = model_ParsBERT
      tokenizer = tokenizer_ParsBERT
    if(model=="Trained ParsBERT"):
      model = trained_bert
      tokenizer = tokenizer_trained_bert  
    # Tokenize the input sentence  
    tokenized_sentence = tokenizer.tokenize("[CLS] " + sentence + "[SEP]")
    mask_index = tokenized_sentence.index("[MASK]")
    token_ids = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    token_tensor = torch.tensor([token_ids])

    # Forward pass without training and backpropagation
    with torch.no_grad():
        outputs = model(token_tensor)
        prediction_logits = outputs[0]
    
    # Calculating 3 nearest prediction of masked token 
    prediction_probs = torch.nn.functional.softmax(prediction_logits[0, mask_index], dim=-1)
    top_k_probs, top_k_token_ids = torch.topk(prediction_probs, 3, sorted=True)

    # Print the predicted tokens for mask and their weight
    predicted_mask = []
    for i in range(3):
        predicted_mask.append(tokenizer.convert_ids_to_tokens([top_k_token_ids[i]])[0])
        token_prob = top_k_probs[i].item()
        print(f"[MASK]: '{predicted_mask[i]}'  : {token_prob}")       
    return predicted_mask[0]   

In [5]:
df["cat"].unique().count()

array(['city', 'capital', 'currency', 'family', 'gram_thirdperson',
       'gram_past', 'gram_firstperson', 'gram_adj2adv', 'gram_noun2adv',
       'gram_antonym', 'gram_comparative', 'gram_nationality',
       'gram_plural'], dtype=object)

 **تست روش اول** 

In [144]:
Accuracy = []
Accuracy_city = 0
y= df[df["cat"]=="city"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "city"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "شهر "+df["first"][i[0]] + " در استان " + df["second"][i[0]] + "  در نتیجه شهر " + df["third"][i[0]] + " در استان " + "[MASK]" + " می باشد."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_city = Accuracy_city + 1   
num = y["cat"].count()      
print(Accuracy_city/num)      



اهواز خوزستان قشم هرمزگان
شهر اهواز در استان خوزستان  در نتیجه شهر قشم در استان [MASK] می باشد.
[MASK]: 'هرمزگان'  : 0.5045045614242554
[MASK]: 'فارس'  : 0.22657565772533417
[MASK]: 'خوزستان'  : 0.09653886407613754
شیراز فارس دامغان سمنان
شهر شیراز در استان فارس  در نتیجه شهر دامغان در استان [MASK] می باشد.
[MASK]: 'سمنان'  : 0.5176708102226257
[MASK]: 'فارس'  : 0.1097259670495987
[MASK]: 'یزد'  : 0.05482158809900284
رشت گیلان اسلام‌شهر تهران
شهر رشت در استان گیلان  در نتیجه شهر اسلام‌شهر در استان [MASK] می باشد.
[MASK]: 'تهران'  : 0.4923759996891022
[MASK]: 'قزوین'  : 0.12632086873054504
[MASK]: 'مرکزی'  : 0.03751932457089424
بندرعباس هرمزگان ملارد تهران
شهر بندرعباس در استان هرمزگان  در نتیجه شهر ملارد در استان [MASK] می باشد.
[MASK]: 'هرمزگان'  : 0.2596472203731537
[MASK]: 'فارس'  : 0.2051641047000885
[MASK]: 'کرمان'  : 0.14770469069480896
اراک مرکزی مریوان کردستان
شهر اراک در استان مرکزی  در نتیجه شهر مریوان در استان [MASK] می باشد.
[MASK]: 'کردستان'  : 0.8301264047622681
[MASK]: '

In [45]:
Accuracy_currency = 0
y= df[df["cat"]=="currency"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "currency"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " واحد پول "+df["second"][i[0]] + " برای کشور " + df["first"][i[0]] + " است پس ارز و پول " + "[MASK]" + " برای کشور " + df["third"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_currency = Accuracy_currency + 1   
num = y["cat"].count()      
print(Accuracy_currency/num)     

سوئد کرون امارات درهم
 واحد پول کرون برای کشور سوئد است پس واحد پول [MASK] برای کشور امارات است.
[MASK]: 'کرون'  : 0.2929396629333496
[MASK]: 'ان'  : 0.18238890171051025
[MASK]: 'یورو'  : 0.05360444262623787
سوئیس فرانک تاجیکستان سامانی
 واحد پول فرانک برای کشور سوئیس است پس واحد پول [MASK] برای کشور تاجیکستان است.
[MASK]: 'ان'  : 0.3017779588699341
[MASK]: 'فرانک'  : 0.06364057213068008
[MASK]: 'تاجیکستان'  : 0.05856305733323097
رومانی لئو بریتانیا پوند
 واحد پول لئو برای کشور رومانی است پس واحد پول [MASK] برای کشور بریتانیا است.
[MASK]: 'ان'  : 0.5437384843826294
[MASK]: 'یورو'  : 0.03399169445037842
[MASK]: 'رسمی'  : 0.02070535719394684
ترکیه لیره افغانستان افغانی
 واحد پول لیره برای کشور ترکیه است پس واحد پول [MASK] برای کشور افغانستان است.
[MASK]: 'ان'  : 0.4961705207824707
[MASK]: 'لیره'  : 0.0951341763138771
[MASK]: 'ترکیه'  : 0.039821356534957886
چین یوان عراق دینار
 واحد پول یوان برای کشور چین است پس واحد پول [MASK] برای کشور عراق است.
[MASK]: 'ان'  : 0.4119029939174652
[MASK]

In [47]:
Accuracy_capital = 0
y= df[df["cat"]=="capital"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "capital"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " شهر "+df["second"][i[0]] + " پایتخت کشور " + df["first"][i[0]] + " است پس شهر " + "[MASK]" + " پایتخت کشور " + df["third"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_capital = Accuracy_capital + 1   
num = y["cat"].count()      
print(Accuracy_capital/num)     

روسیه مسکو هلند آمستردام
 شهر مسکو پایتخت کشور روسیه است پس شهر [MASK] پایتخت کشور هلند است.
[MASK]: 'امستردام'  : 0.4687177538871765
[MASK]: 'روتردام'  : 0.12137912213802338
[MASK]: 'بروکسل'  : 0.03916442021727562
کره‌جنوبی سئول آمریکا واشنگتن
 شهر سئول پایتخت کشور کره‌جنوبی است پس شهر [MASK] پایتخت کشور آمریکا است.
[MASK]: 'واشنگتن'  : 0.22029808163642883
[MASK]: 'نیویورک'  : 0.14643797278404236
[MASK]: 'سانفرانسیسکو'  : 0.10626213252544403
ایران تهران اسپانیا مادرید
 شهر تهران پایتخت کشور ایران است پس شهر [MASK] پایتخت کشور اسپانیا است.
[MASK]: 'مادرید'  : 0.3078828752040863
[MASK]: 'بارسلونا'  : 0.12862356007099152
[MASK]: 'سویل'  : 0.10865620523691177
ایران تهران افغانستان کابل
 شهر تهران پایتخت کشور ایران است پس شهر [MASK] پایتخت کشور افغانستان است.
[MASK]: 'کابل'  : 0.3628406524658203
[MASK]: 'مزارشریف'  : 0.2060290426015854
[MASK]: 'هرات'  : 0.1943739652633667
بریتانیا لندن فنلاند هلسینکی
 شهر لندن پایتخت کشور بریتانیا است پس شهر [MASK] پایتخت کشور فنلاند است.
[MASK]: 'هلسینکی'

In [53]:
Accuracy_family = 0
y= df[df["cat"]=="family"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "family"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت خانوادگی "+df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت خانوادگی " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_family = Accuracy_family + 1   
num = y["cat"].count()      
print(Accuracy_family/num)     

برادر خواهر پدر مادر
 نسبت خانوادگی برادر به خواهر مانند نسبت خانوادگی پدر به [MASK] است.
[MASK]: 'مادر'  : 0.6692098379135132
[MASK]: 'برادر'  : 0.09877248853445053
[MASK]: 'پسر'  : 0.07660110294818878
برادرها خواهرها شوهر عیال
 نسبت خانوادگی برادرها به خواهرها مانند نسبت خانوادگی شوهر به [MASK] است.
[MASK]: 'زن'  : 0.1912340372800827
[MASK]: 'خواهر'  : 0.14468593895435333
[MASK]: 'پدر'  : 0.11019666492938995
پدربزرگ مادربزرگ برادر خواهر
 نسبت خانوادگی پدربزرگ به مادربزرگ مانند نسبت خانوادگی برادر به [MASK] است.
[MASK]: 'خواهر'  : 0.40978193283081055
[MASK]: 'پدر'  : 0.23004050552845
[MASK]: 'پدربزرگ'  : 0.1903122216463089
بابابزرگ مامان‌بزرگ داماد عروس
 نسبت خانوادگی بابابزرگ به مامان‌بزرگ مانند نسبت خانوادگی داماد به [MASK] است.
[MASK]: 'عروس'  : 0.2780780792236328
[MASK]: 'داماد'  : 0.17299982905387878
[MASK]: 'خاله'  : 0.054446108639240265
داماد عروس برادرها خواهرها
 نسبت خانوادگی داماد به عروس مانند نسبت خانوادگی برادرها به [MASK] است.
[MASK]: 'داماد'  : 0.2346702665090561
[MASK]

In [61]:
Accuracy = 0
y= df[df["cat"]=="gram_thirdperson"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_thirdperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " فعل "+df["first"][i[0]] + " سوم شخص مفرد و فعل " + df["second"][i[0]] + " سوم شخص جمع است در نتیجه فعل " + df["third"][i[0]] + " سوم شخص مفرد و فعل " + "[MASK]" + " سوم شخص جمع است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)     

برید بریدند رقصید رقصیدند
 فعل برید سوم شخص مفرد و فعل بریدند سوم شخص جمع است در نتیجه فعل رقصید سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'برید'  : 0.049297038465738297
[MASK]: 'جمع'  : 0.035543475300073624
[MASK]: 'غیر'  : 0.025543291121721268
پرید پریدند آمد آمدند
 فعل پرید سوم شخص مفرد و فعل پریدند سوم شخص جمع است در نتیجه فعل آمد سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'امد'  : 0.7506507039070129
[MASK]: 'برگشت'  : 0.015064816921949387
[MASK]: 'اورد'  : 0.01407806295901537
تازید تازیدند گفت گفتند
 فعل تازید سوم شخص مفرد و فعل تازیدند سوم شخص جمع است در نتیجه فعل گفت سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'تازی'  : 0.12695050239562988
[MASK]: 'گفت'  : 0.07075846195220947
[MASK]: 'عجم'  : 0.023389030247926712
شد شدند بوسید بوسیدند
 فعل شد سوم شخص مفرد و فعل شدند سوم شخص جمع است در نتیجه فعل بوسید سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'شد'  : 0.16097494959831238
[MASK]: 'دوم'  : 0.048131536692380905
[MASK]: 'جمع'  : 0.02071278728544712
کرد

In [86]:
Accuracy = 0
y= df[df["cat"]=="gram_past"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_past"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "  مصدر "+df["third"][i[0]] + " فعل " + "[MASK]" + " است همانطور که مصدر " + df["first"][i[0]] + " فعل " + df["second"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)      

بریدن برید دمیدن دمید
  مصدر دمیدن فعل [MASK] است همانطور که مصدر بریدن فعل برید است.
[MASK]: 'دمیدن'  : 0.07907748967409134
[MASK]: 'کشیدن'  : 0.06370734423398972
[MASK]: 'بریدن'  : 0.061953429132699966
تازیدن تاخت دادن داد
  مصدر دادن فعل [MASK] است همانطور که مصدر تازیدن فعل تاخت است.
[MASK]: 'تازی'  : 0.12566618621349335
[MASK]: 'تاخت'  : 0.02383355237543583
[MASK]: 'مصدر'  : 0.022929402068257332
شدن شد نگریستن نگریست
  مصدر نگریستن فعل [MASK] است همانطور که مصدر شدن فعل شد است.
[MASK]: 'شد'  : 0.43245089054107666
[MASK]: 'شدن'  : 0.07902842015028
[MASK]: 'کرد'  : 0.04641430452466011
گفتن گفت رقصیدن رقصید
  مصدر رقصیدن فعل [MASK] است همانطور که مصدر گفتن فعل گفت است.
[MASK]: 'گفتن'  : 0.44676876068115234
[MASK]: 'زدن'  : 0.03163199499249458
[MASK]: 'گفت'  : 0.02703920379281044
گفتن گفت خواستن خواست
  مصدر خواستن فعل [MASK] است همانطور که مصدر گفتن فعل گفت است.
[MASK]: 'گفتن'  : 0.4737354516983032
[MASK]: 'خواستن'  : 0.14876383543014526
[MASK]: 'دادن'  : 0.03337039425969124
توانستن 

In [84]:
Accuracy = 0
y= df[df["cat"]=="gram_firstperson"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_firstperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " من "+df["third"][i[0]] + "  پس ما " + "[MASK]" + " من " + df["first"][i[0]] + " و ما " + df["second"][i[0]] + " "
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

دادم دادیم یافتم یافتیم
 من یافتم  پس ما [MASK] من دادم و ما دادیم 
[MASK]: 'به'  : 0.6317965984344482
[MASK]: 'را'  : 0.07148614525794983
[MASK]: 'دادیم'  : 0.04641858860850334
زدم زدیم نامیدم نامیدیم
 من نامیدم  پس ما [MASK] من زدم و ما زدیم 
[MASK]: 'را'  : 0.22065837681293488
[MASK]: 'زدیم'  : 0.0799311175942421
[MASK]: 'و'  : 0.05988111346960068
نوشتم نوشتیم گفتم گفتیم
 من گفتم  پس ما [MASK] من نوشتم و ما نوشتیم 
[MASK]: 'نوشتیم'  : 0.3631836473941803
[MASK]: 'و'  : 0.08693627268075943
[MASK]: 'نوشتم'  : 0.07703155279159546
دانستم دانستیم بلعیدم بلعیدیم
 من بلعیدم  پس ما [MASK] من دانستم و ما دانستیم 
[MASK]: 'دانستیم'  : 0.6740878224372864
[MASK]: 'را'  : 0.1716470867395401
[MASK]: '،'  : 0.022580716758966446
گردیدم گردیدیم یافتم یافتیم
 من یافتم  پس ما [MASK] من گردیدم و ما گردیدیم 
[MASK]: 'یافتیم'  : 0.13374878466129303
[MASK]: 'و'  : 0.09252305328845978
[MASK]: '،'  : 0.08479662984609604
گردیدم گردیدیم بوسیدم بوسیدیم
 من بوسیدم  پس ما [MASK] من گردیدم و ما گردیدیم 
[MASK]: 'و

In [90]:
Accuracy = 0
y= df[df["cat"]=="gram_adj2adv"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_adj2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "  "+ df["first"][i[0]] + " صفت و " + df["second"][i[0]] + " قید است. در نتیجه " + df["third"][i[0]] + " صفت و " + "[MASK]" + " قید است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

مجدد مجددا مخصوص مخصوصا
  مجدد صفت و مجددا قید است. در نتیجه مخصوص صفت و [MASK] قید است.
[MASK]: 'مجددا'  : 0.6083488464355469
[MASK]: 'دوباره'  : 0.10037857294082642
[MASK]: 'مجدد'  : 0.07224800437688828
عمده عمدتا صمیمی صمیمانه
  عمده صفت و عمدتا قید است. در نتیجه صمیمی صفت و [MASK] قید است.
[MASK]: 'عمدتا'  : 0.35317128896713257
[MASK]: 'غالبا'  : 0.23248420655727386
[MASK]: 'عموما'  : 0.04988967999815941
دقیق دقیقا عادل عادلانه
  دقیق صفت و دقیقا قید است. در نتیجه عادل صفت و [MASK] قید است.
[MASK]: 'درست'  : 0.07977661490440369
[MASK]: 'دقیق'  : 0.04434031620621681
[MASK]: 'دقیقا'  : 0.030385633930563927
غالب غالبا جدا جداگانه
  غالب صفت و غالبا قید است. در نتیجه جدا صفت و [MASK] قید است.
[MASK]: 'غالب'  : 0.5035934448242188
[MASK]: 'غالبا'  : 0.15999457240104675
[MASK]: 'مغلوب'  : 0.030084261670708656
متناظر متناظرا قبل قبلا
  متناظر صفت و متناظرا قید است. در نتیجه قبل صفت و [MASK] قید است.
[MASK]: 'متناظر'  : 0.8398282527923584
[MASK]: 'بعد'  : 0.036908697336912155
[MASK]: 'قبل' 

In [94]:
Accuracy = 0
y= df[df["cat"]=="gram_noun2adv"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_noun2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

اتفاق اتفاقا شب شبانه
 نسبت اتفاق به اتفاقا مانند نسبت شب به [MASK] است.
[MASK]: 'روز'  : 0.5458536744117737
[MASK]: 'ماه'  : 0.03814972564578056
[MASK]: 'ظهر'  : 0.022772541269659996
حتم حتما اساس اساسا
 نسبت حتم به حتما مانند نسبت اساس به [MASK] است.
[MASK]: 'اساس'  : 0.06901869922876358
[MASK]: 'فرد'  : 0.0281374529004097
[MASK]: 'مبنا'  : 0.021187210455536842
اصل اصلا انسان انسانوار
 نسبت اصل به اصلا مانند نسبت انسان به [MASK] است.
[MASK]: 'طبیعت'  : 0.08071651309728622
[MASK]: 'واقعیت'  : 0.0740446001291275
[MASK]: 'حقیقت'  : 0.05440415441989899
عمل عملا اجبار اجبارا
 نسبت عمل به عملا مانند نسبت اجبار به [MASK] است.
[MASK]: 'عمل'  : 0.19409328699111938
[MASK]: 'اجبار'  : 0.17755594849586487
[MASK]: 'تسلیم'  : 0.026661230251193047
اجبار اجبارا کودک کودکانه
 نسبت اجبار به اجبارا مانند نسبت کودک به [MASK] است.
[MASK]: 'مادر'  : 0.12338463962078094
[MASK]: 'والدین'  : 0.0715404599905014
[MASK]: 'معلم'  : 0.05993014574050903
دائم دائما اتفاق اتفاقا
 نسبت دائم به دائما مانند نسبت اتفاق 

In [104]:
Accuracy = 0
y= df[df["cat"]=="gram_antonym"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_antonym"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

خشنود ناخشنود موفق ناموفق
خشنود به ناخشنود مانند موفق به [MASK] است.
[MASK]: 'خشنود'  : 0.6726464629173279
[MASK]: 'ناخشنود'  : 0.050517186522483826
[MASK]: 'راضی'  : 0.007278201170265675
درست نادرست پخته ناپخته
درست به نادرست مانند پخته به [MASK] است.
[MASK]: 'غلط'  : 0.13538186252117157
[MASK]: 'اشتباه'  : 0.05775130167603493
[MASK]: 'درست'  : 0.03680913522839546
مناسب نامناسب همزمان ناهمزمان
مناسب به نامناسب مانند همزمان به [MASK] است.
[MASK]: 'هم'  : 0.0608019158244133
[MASK]: 'پایین'  : 0.021565567702054977
[MASK]: 'یک'  : 0.012066750787198544
مربوط نامربوط منسجم نامنسجم
مربوط به نامربوط مانند منسجم به [MASK] است.
[MASK]: 'خود'  : 0.03065207600593567
[MASK]: 'هم'  : 0.027194123715162277
[MASK]: 'متن'  : 0.017043383792042732
بینا نابینا درست نادرست
بینا به نابینا مانند درست به [MASK] است.
[MASK]: 'نابینا'  : 0.31026366353034973
[MASK]: 'بینا'  : 0.057476725429296494
[MASK]: 'شنوا'  : 0.046706221997737885
کارآمد ناکارآمد هنجار ناهنجار
کارآمد به ناکارآمد مانند هنجار به [MASK] است.
[M

In [110]:
Accuracy = 0
y= df[df["cat"]=="gram_comparative"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_comparative"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

سریع سریعتر بلند بلندتر
 نسبت سریع به سریعتر مانند نسبت بلند به [MASK] است.
[MASK]: 'کوتاه'  : 0.2847450077533722
[MASK]: 'سریع'  : 0.1590980738401413
[MASK]: 'اهسته'  : 0.038366448134183884
بزرگ بزرگتر شدید شدیدتر
 نسبت بزرگ به بزرگتر مانند نسبت شدید به [MASK] است.
[MASK]: 'بزرگتر'  : 0.1062101423740387
[MASK]: 'بزرگ'  : 0.09587859362363815
[MASK]: 'متوسط'  : 0.058760933578014374
مهم مهمتر بارز بارزتر
 نسبت مهم به مهمتر مانند نسبت بارز به [MASK] است.
[MASK]: 'بارز'  : 0.195845827460289
[MASK]: 'مهمتر'  : 0.09522134065628052
[MASK]: 'برجسته'  : 0.046095941215753555
به بهتر سرد سردتر
 نسبت به به بهتر مانند نسبت سرد به [MASK] است.
[MASK]: 'گرم'  : 0.42213332653045654
[MASK]: 'بهتر'  : 0.040560539811849594
[MASK]: 'خشک'  : 0.03984379768371582
واضح واضحتر بیش بیشتر
 نسبت واضح به واضحتر مانند نسبت بیش به [MASK] است.
[MASK]: 'واضح'  : 0.05552274361252785
[MASK]: 'واضحتر'  : 0.03452478349208832
[MASK]: 'صفر'  : 0.02421565353870392
کم کمتر بد بدتر
 نسبت کم به کمتر مانند نسبت بد به [MASK] است.


In [121]:
Accuracy = 0
y= df[df["cat"]=="gram_nationality"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_nationality"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " مردم " + df["second"][i[0]] + " اهل کشور " + df["first"][i[0]] + " هستند. و مردم " + "[MASK]" + " اهل کشور " + df["third"][i[0]] + " هستند."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

اتریش اتریشی اسپانیا اسپانیایی
 مردم اتریشی اهل کشور اتریش هستند. و مردم [MASK] اهل کشور اسپانیا هستند.
[MASK]: 'اسپانیایی'  : 0.18227513134479523
[MASK]: 'اسپانیا'  : 0.13247904181480408
[MASK]: 'کاتالان'  : 0.05533662065863609
سوئد سوئدی فنلاند فنلاندی
 مردم سوئدی اهل کشور سوئد هستند. و مردم [MASK] اهل کشور فنلاند هستند.
[MASK]: 'فنلاندی'  : 0.3014686107635498
[MASK]: 'سويدی'  : 0.19402451813220978
[MASK]: 'نروژی'  : 0.183609738945961
بحرین بحرینی عراق عراقی
 مردم بحرینی اهل کشور بحرین هستند. و مردم [MASK] اهل کشور عراق هستند.
[MASK]: 'بحرین'  : 0.35381653904914856
[MASK]: 'عراق'  : 0.12622299790382385
[MASK]: 'بحرینی'  : 0.052589479833841324
کویت کویتی سوریه سوری
 مردم کویتی اهل کشور کویت هستند. و مردم [MASK] اهل کشور سوریه هستند.
[MASK]: 'کویتی'  : 0.6979068517684937
[MASK]: 'سوری'  : 0.09482868760824203
[MASK]: 'سوریه'  : 0.02939140982925892
مصر مصری کرواسی کروات
 مردم مصری اهل کشور مصر هستند. و مردم [MASK] اهل کشور کرواسی هستند.
[MASK]: 'کرواسی'  : 0.06758736819028854
[MASK]: 'سو

In [133]:
Accuracy = 0
y= df[df["cat"]=="gram_plural"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_plural"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " " + df["first"][i[0]] + " جمع " + df["second"][i[0]] + " است و " + df["third"][i[0]] + " جمع " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

تالیف تالیفات مخلوق مخلوقات
 تالیف جمع تالیفات است و مخلوق جمع [MASK] است.
[MASK]: 'تالیف'  : 0.25174930691719055
[MASK]: 'کتاب'  : 0.0738530084490776
[MASK]: 'اثار'  : 0.04923452064394951
اثر اثرات ارزش ارزشها
 اثر جمع اثرات است و ارزش جمع [MASK] است.
[MASK]: 'اثر'  : 0.25880610942840576
[MASK]: 'اثرات'  : 0.08273234218358994
[MASK]: 'ارزش'  : 0.054366689175367355
انقلابی انقلابیون مورد موارد
 انقلابی جمع انقلابیون است و مورد جمع [MASK] است.
[MASK]: 'انقلابیون'  : 0.23702497780323029
[MASK]: 'مردم'  : 0.2052450329065323
[MASK]: 'انقلاب'  : 0.03541312739253044
حواری حواریون مورد موارد
 حواری جمع حواریون است و مورد جمع [MASK] است.
[MASK]: 'حواریون'  : 0.469120055437088
[MASK]: 'رسولان'  : 0.03176255151629448
[MASK]: 'بهاءالله'  : 0.014634152874350548
مسئول مسئولین آزاده آزادگان
 مسئول جمع مسئولین است و آزاده جمع [MASK] است.
[MASK]: 'مردم'  : 0.11778392642736435
[MASK]: 'مسيولین'  : 0.08105282485485077
[MASK]: 'افراد'  : 0.07055479288101196
مسافر مسافرین طلا طلاها
 مسافر جمع مسافرین است 

**تست روش دوم**

In [12]:
Accuracy= 0
y= df[df["cat"]=="city"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "city"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " "+df["first"][i[0]] + " در " + "[MASK]" + df["second"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = " "+df["third"][i[0]] + " در " + predicted + "[MASK]" + " است."
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)      


اهواز خوزستان قشم هرمزگان
 اهواز در [MASK]خوزستان است.
[MASK]: 'استان'  : 0.9299547076225281
[MASK]: 'جنوب'  : 0.015550939366221428
[MASK]: 'غرب'  : 0.009022115729749203
 قشم در استان[MASK] است.
[MASK]: 'هرمزگان'  : 0.8786002397537231
[MASK]: 'قشم'  : 0.05819853022694588
[MASK]: 'فارس'  : 0.02707228995859623
شیراز فارس دامغان سمنان
 شیراز در [MASK]فارس است.
[MASK]: 'استان'  : 0.764687716960907
[MASK]: 'جنوب'  : 0.059331346303224564
[MASK]: 'شمال'  : 0.02427971549332142
 دامغان در استان[MASK] است.
[MASK]: 'سمنان'  : 0.6204450130462646
[MASK]: 'دامغان'  : 0.15624210238456726
[MASK]: 'خراسان'  : 0.08957114070653915
رشت گیلان اسلام‌شهر تهران
 رشت در [MASK]گیلان است.
[MASK]: 'استان'  : 0.8420323729515076
[MASK]: 'غرب'  : 0.03320040926337242
[MASK]: 'جنوب'  : 0.02603726275265217
 اسلام‌شهر در استان[MASK] است.
[MASK]: 'تهران'  : 0.5965381264686584
[MASK]: 'کرمان'  : 0.13670730590820312
[MASK]: 'قزوین'  : 0.03208678215742111
بندرعباس هرمزگان ملارد تهران
 بندرعباس در [MASK]هرمزگان است.
[MASK]: 

In [19]:
Accuracy= 0
y= df[df["cat"]=="capital"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "capital"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = " "+"[MASK]" + " " + predicted + " " + df["third"][i[0]] + " است."
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)   

روسیه مسکو هلند آمستردام
مسکو [MASK] روسیه است.
[MASK]: 'پایتخت'  : 0.8754922747612
[MASK]: 'قلب'  : 0.014657219871878624
[MASK]: 'شهر'  : 0.010168214328587055
 [MASK] پایتخت هلند است.
[MASK]: 'امستردام'  : 0.35550057888031006
[MASK]: 'روتردام'  : 0.17388218641281128
[MASK]: 'لیدن'  : 0.15383394062519073
کره‌جنوبی سئول آمریکا واشنگتن
سئول [MASK] کره‌جنوبی است.
[MASK]: 'پایتخت'  : 0.9711080193519592
[MASK]: 'مرکز'  : 0.00899849645793438
[MASK]: 'شهر'  : 0.0028767092153429985
 [MASK] پایتخت آمریکا است.
[MASK]: 'نیویورک'  : 0.3681582510471344
[MASK]: 'لسانجلس'  : 0.0504448227584362
[MASK]: 'واشینگتن'  : 0.04400761052966118
ایران تهران اسپانیا مادرید
تهران [MASK] ایران است.
[MASK]: 'پایتخت'  : 0.9565833210945129
[MASK]: 'مرکز'  : 0.008535626344382763
[MASK]: 'شهر'  : 0.003112700767815113
 [MASK] پایتخت اسپانیا است.
[MASK]: 'مادرید'  : 0.313161164522171
[MASK]: 'بارسلونا'  : 0.2423686534166336
[MASK]: 'تولدو'  : 0.05668731778860092
ایران تهران افغانستان کابل
تهران [MASK] ایران است.
[MASK]: 

In [25]:
Accuracy= 0
y= df[df["cat"]=="currency"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "currency"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است."
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)   

سوئد کرون امارات درهم
کرون [MASK] سوئد است.
[MASK]: 'کرون'  : 0.2224176675081253
[MASK]: 'کشور'  : 0.18625760078430176
[MASK]: 'در'  : 0.056126050651073456
کرونامارات [MASK] است.
[MASK]: 'نادر'  : 0.04760163649916649
[MASK]: 'نامشخص'  : 0.031456850469112396
[MASK]: 'ناشناخته'  : 0.030047010630369186
سوئیس فرانک تاجیکستان سامانی
فرانک [MASK] سوئیس است.
[MASK]: 'کشور'  : 0.10573209822177887
[MASK]: 'نماد'  : 0.08561397343873978
[MASK]: 'فرانک'  : 0.07495801895856857
کشورتاجیکستان [MASK] است.
[MASK]: 'ایران'  : 0.048310671001672745
[MASK]: 'مصر'  : 0.021053623408079147
[MASK]: 'کوچک'  : 0.020535539835691452
رومانی لئو بریتانیا پوند
لئو [MASK] رومانی است.
[MASK]: 'متولد'  : 0.5341964960098267
[MASK]: 'اهل'  : 0.28928789496421814
[MASK]: 'زاده'  : 0.09764347225427628
متولدبریتانیا [MASK] است.
[MASK]: 'کاتولیک'  : 0.02463184855878353
[MASK]: 'فرانسوی'  : 0.02080654352903366
[MASK]: 'فرانسه'  : 0.01874217391014099
ترکیه لیره افغانستان افغانی
لیره [MASK] ترکیه است.
[MASK]: 'ایران'  : 0.0788392

In [29]:
Accuracy= 0
y= df[df["cat"]=="family"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "family"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

برادر خواهر پدر مادر
خواهر [MASK] برادر است.
[MASK]: 'او'  : 0.19088268280029297
[MASK]: 'وی'  : 0.06992656737565994
[MASK]: 'دیگر'  : 0.03568146005272865
اوپدر [MASK] است. 
[MASK]: 'دسترس'  : 0.02265206351876259
[MASK]: 'اینجا'  : 0.010621064342558384
[MASK]: 'راه'  : 0.01043678354471922
برادرها خواهرها شوهر عیال
خواهرها [MASK] برادرها است.
[MASK]: 'و'  : 0.11399473249912262
[MASK]: 'شبیه'  : 0.08395306020975113
[MASK]: 'از'  : 0.06611459702253342
وشوهر [MASK] است. 
[MASK]: 'خوب'  : 0.031019974499940872
[MASK]: 'زن'  : 0.01587618701159954
[MASK]: 'مرد'  : 0.010337420739233494
پدربزرگ مادربزرگ برادر خواهر
مادربزرگ [MASK] پدربزرگ است.
[MASK]: 'او'  : 0.2762252390384674
[MASK]: 'وی'  : 0.073356032371521
[MASK]: 'من'  : 0.04295407980680466
اوبرادر [MASK] است. 
[MASK]: 'اوبرا'  : 0.01985454373061657
[MASK]: 'خوب'  : 0.01507636159658432
[MASK]: 'همین'  : 0.01425120234489441
بابابزرگ مامان‌بزرگ داماد عروس
مامان‌بزرگ [MASK] بابابزرگ است.
[MASK]: 'شبیه'  : 0.13705801963806152
[MASK]: 'همان'  :

In [32]:
Accuracy= 0
y= df[df["cat"]=="gram_thirdperson"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_thirdperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["second"][i[0]] + " به " +df["first"][i[0]]  + " " + "[MASK]"+ " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

برید بریدند رقصید رقصیدند
 نسبت بریدند به برید [MASK] است.
[MASK]: 'متفاوت'  : 0.24139787256717682
[MASK]: 'یکسان'  : 0.04237256571650505
[MASK]: 'بیشتر'  : 0.034068163484334946
متفاوترقصید [MASK] است. 
[MASK]: 'متفاوت'  : 0.0626164898276329
[MASK]: 'ان'  : 0.032515328377485275
[MASK]: 'زیاد'  : 0.0250974353402853
پرید پریدند آمد آمدند
 نسبت پریدند به پرید [MASK] است.
[MASK]: 'صفر'  : 0.05362393334507942
[MASK]: '۱'  : 0.04759844020009041
[MASK]: '۲'  : 0.04750465974211693
صفرآمد [MASK] است. 
[MASK]: 'صفرا'  : 0.050644401460886
[MASK]: 'خون'  : 0.03450227528810501
[MASK]: 'ارثی'  : 0.03273545578122139
تازید تازیدند گفت گفتند
 نسبت تازیدند به تازید [MASK] است.
[MASK]: 'متفاوت'  : 0.1997176557779312
[MASK]: 'اندک'  : 0.048609115183353424
[MASK]: 'یکسان'  : 0.031012730672955513
متفاوتگفت [MASK] است. 
[MASK]: 'زدن'  : 0.0827072411775589
[MASK]: 'گو'  : 0.059098728001117706
[MASK]: 'ناپذیر'  : 0.04322291538119316
شد شدند بوسید بوسیدند
 نسبت شدند به شد [MASK] است.
[MASK]: 'متفاوت'  : 0.14271

In [33]:
Accuracy= 0
y= df[df["cat"]=="gram_past"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_past"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

بریدن برید دمیدن دمید
برید [MASK] بریدن است.
[MASK]: '،'  : 0.19749504327774048
[MASK]: 'همان'  : 0.1643521785736084
[MASK]: 'و'  : 0.055184122174978256
،دمیدن [MASK] است. 
[MASK]: 'نفس'  : 0.03066079132258892
[MASK]: 'لازم'  : 0.028200611472129822
[MASK]: 'ضروری'  : 0.0187393669039011
تازیدن تاخت دادن داد
تاخت [MASK] تازیدن است.
[MASK]: '،'  : 0.18445952236652374
[MASK]: 'و'  : 0.10940387845039368
[MASK]: 'همان'  : 0.06523173302412033
،دادن [MASK] است. 
[MASK]: 'خوب'  : 0.03450595214962959
[MASK]: 'حق'  : 0.03053072839975357
[MASK]: 'وظیفه'  : 0.02220144495368004
شدن شد نگریستن نگریست
شد [MASK] شدن است.
[MASK]: 'بزرگ'  : 0.027858564630150795
[MASK]: 'تمام'  : 0.02087506651878357
[MASK]: 'عوض'  : 0.019519714638590813
بزرگنگریستن [MASK] است. 
[MASK]: 'مهم'  : 0.028944360092282295
[MASK]: 'هنر'  : 0.025917785242199898
[MASK]: 'ضروری'  : 0.018199589103460312
گفتن گفت رقصیدن رقصید
گفت [MASK] گفتن است.
[MASK]: 'سخن'  : 0.08632168918848038
[MASK]: ':'  : 0.05512664467096329
[MASK]: 'دروغ'  :

In [34]:
Accuracy= 0
y= df[df["cat"]=="gram_firstperson"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_firstperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

دادم دادیم یافتم یافتیم
دادیم [MASK] دادم است.
[MASK]: 'و'  : 0.21605373919010162
[MASK]: '،'  : 0.07398651540279388
[MASK]: 'که'  : 0.048536673188209534
ویافتم [MASK] است. 
[MASK]: 'کوتاه'  : 0.012853864580392838
[MASK]: 'برابر'  : 0.009119750000536442
[MASK]: 'دوم'  : 0.006297701969742775
زدم زدیم نامیدم نامیدیم
زدیم [MASK] زدم است.
[MASK]: 'حرف'  : 0.18320854008197784
[MASK]: 'و'  : 0.05485466122627258
[MASK]: 'که'  : 0.039328888058662415
حرفنامیدم [MASK] است. 
[MASK]: 'راست'  : 0.04268628731369972
[MASK]: 'فارسی'  : 0.025003138929605484
[MASK]: 'فتحه'  : 0.02039364166557789
نوشتم نوشتیم گفتم گفتیم
نوشتیم [MASK] نوشتم است.
[MASK]: 'و'  : 0.3429446816444397
[MASK]: '،'  : 0.30682286620140076
[MASK]: '.'  : 0.12457075715065002
وگفتم [MASK] است. 
[MASK]: 'خوب'  : 0.024466505274176598
[MASK]: 'دروغ'  : 0.020243510603904724
[MASK]: 'درست'  : 0.019946588203310966
دانستم دانستیم بلعیدم بلعیدیم
دانستیم [MASK] دانستم است.
[MASK]: 'و'  : 0.3065733313560486
[MASK]: '،'  : 0.19485777616500854
[

In [35]:
Accuracy= 0
y= df[df["cat"]=="gram_adj2adv"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_adj2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

مجدد مجددا مخصوص مخصوصا
مجددا [MASK] مجدد است.
[MASK]: 'انتخاب'  : 0.051064182072877884
[MASK]: 'شروع'  : 0.02827024832367897
[MASK]: 'و'  : 0.015785949304699898
انتخابمخصوص [MASK] است. 
[MASK]: 'اختیاری'  : 0.14822812378406525
[MASK]: 'اسان'  : 0.08052608370780945
[MASK]: 'مهم'  : 0.035108037292957306
عمده عمدتا صمیمی صمیمانه
عمدتا [MASK] عمده است.
[MASK]: 'کار'  : 0.01609984040260315
[MASK]: 'مشتری'  : 0.015754621475934982
[MASK]: 'تولیدکننده'  : 0.011743574403226376
کارصمیمی [MASK] است. 
[MASK]: 'خوب'  : 0.12200551480054855
[MASK]: 'مهم'  : 0.06319628655910492
[MASK]: 'هم'  : 0.03397791460156441
دقیق دقیقا عادل عادلانه
دقیقا [MASK] دقیق است.
[MASK]: 'خیلی'  : 0.157426655292511
[MASK]: 'چقدر'  : 0.1149527058005333
[MASK]: 'هم'  : 0.09604846686124802
خیلیعادل [MASK] است. 
[MASK]: '##ه'  : 0.05545685440301895
[MASK]: '##پذیر'  : 0.04555348679423332
[MASK]: '##عادل'  : 0.02778751216828823
غالب غالبا جدا جداگانه
غالبا [MASK] غالب است.
[MASK]: 'تبلیغات'  : 0.016044866293668747
[MASK]: 'شع

In [36]:
Accuracy= 0
y= df[df["cat"]=="gram_noun2adv"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_noun2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

[MASK]: 'مهم'  : 0.010222692042589188
[MASK]: 'بوده'  : 0.00743188988417387
[MASK]: 'بزرگ'  : 0.00635677482932806
حتم حتما اساس اساسا
حتما [MASK] حتم است.
[MASK]: 'هم'  : 0.11569847911596298
[MASK]: 'نتیجه'  : 0.0780905932188034
[MASK]: 'جواب'  : 0.06721463054418564
هماساس [MASK] است. 
[MASK]: 'بیمار'  : 0.017235727980732918
[MASK]: 'بردبار'  : 0.013140847906470299
[MASK]: 'معتقد'  : 0.012617855332791805
اصل اصلا انسان انسانوار
اصلا [MASK] اصل است.
[MASK]: 'این'  : 0.24587616324424744
[MASK]: 'اینها'  : 0.023653585463762283
[MASK]: 'یک'  : 0.02137005142867565
اینانسان [MASK] است. 
[MASK]: 'چنین'  : 0.019872523844242096
[MASK]: 'بسیار'  : 0.018741698935627937
[MASK]: 'خوب'  : 0.013328815810382366
عمل عملا اجبار اجبارا
عملا [MASK] عمل است.
[MASK]: 'ملاک'  : 0.12522770464420319
[MASK]: 'در'  : 0.06983578205108643
[MASK]: 'قابل'  : 0.04486961290240288
ملاکاجبار [MASK] است. 
[MASK]: 'مالزی'  : 0.028946146368980408
[MASK]: 'معروف'  : 0.023525197058916092
[MASK]: 'امروزی'  : 0.020275026559829

In [41]:
Accuracy= 0
y= df[df["cat"]=="gram_antonym"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_antonym"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["first"][i[0]] + " و " + df["second"][i[0]]+"[MASK]" + " یکدیگر هستند."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

خشنود ناخشنود موفق ناموفق
خشنود و ناخشنود[MASK] یکدیگر هستند.
[MASK]: 'از'  : 0.42166006565093994
[MASK]: 'عاشق'  : 0.05445351079106331
[MASK]: 'مغلوب'  : 0.02693176455795765
ازموفق [MASK] است. 
[MASK]: 'شده'  : 0.6334328055381775
[MASK]: 'نشده'  : 0.03100782260298729
[MASK]: 'خودم'  : 0.02315613441169262
درست نادرست پخته ناپخته
درست و نادرست[MASK] یکدیگر هستند.
[MASK]: 'مکمل'  : 0.27680596709251404
[MASK]: 'مخالف'  : 0.06301378458738327
[MASK]: 'دشمن'  : 0.06234549358487129
مکملپخته [MASK] است. 
[MASK]: 'اب'  : 0.04811816290020943
[MASK]: 'شده'  : 0.03298230841755867
[MASK]: 'سبزیجات'  : 0.030845869332551956
مناسب نامناسب همزمان ناهمزمان
مناسب و نامناسب[MASK] یکدیگر هستند.
[MASK]: 'با'  : 0.2583514451980591
[MASK]: 'برای'  : 0.13220329582691193
[MASK]: 'از'  : 0.09115247428417206
باهمزمان [MASK] است. 
[MASK]: 'متفاوت'  : 0.07544521987438202
[MASK]: 'خوب'  : 0.06384123861789703
[MASK]: 'مهم'  : 0.02861068956553936
مربوط نامربوط منسجم نامنسجم
مربوط و نامربوط[MASK] یکدیگر هستند.
[MASK]: 

In [43]:
Accuracy= 0
y= df[df["cat"]=="gram_comparative"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_comparative"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+ "[MASK]" + " "   +  df["third"][i[0]] + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

سریع سریعتر بلند بلندتر
سریعتر [MASK] سریع است.
[MASK]: 'و'  : 0.4957662522792816
[MASK]: 'از'  : 0.1009853258728981
[MASK]: '،'  : 0.090695321559906
و[MASK] بلند است. 
[MASK]: 'بسیار'  : 0.04322332143783569
[MASK]: 'صدایش'  : 0.020623259246349335
[MASK]: 'سرش'  : 0.02033526636660099
بزرگ بزرگتر شدید شدیدتر
بزرگتر [MASK] بزرگ است.
[MASK]: 'از'  : 0.5658529996871948
[MASK]: '،'  : 0.08349695801734924
[MASK]: 'و'  : 0.06228860467672348
از[MASK] شدید است. 
[MASK]: 'تب'  : 0.07218611240386963
[MASK]: 'نوع'  : 0.05304538831114769
[MASK]: 'دید'  : 0.04379662126302719
مهم مهمتر بارز بارزتر
مهمتر [MASK] مهم است.
[MASK]: 'بودن'  : 0.22915612161159515
[MASK]: 'از'  : 0.13589130342006683
[MASK]: '،'  : 0.1326351910829544
بودن[MASK] بارز است. 
[MASK]: 'بسیار'  : 0.07149426639080048
[MASK]: 'نیز'  : 0.06073104590177536
[MASK]: 'همیشه'  : 0.03617071360349655
به بهتر سرد سردتر
بهتر [MASK] به است.
[MASK]: 'از'  : 0.6126387119293213
[MASK]: 'است'  : 0.0650453269481659
[MASK]: 'هم'  : 0.0213818550109863

In [48]:
Accuracy= 0
y= df[df["cat"]=="gram_nationality"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_nationality"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+predicted+" "+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

اتریش اتریشی اسپانیا اسپانیایی
اتریشی [MASK] اتریش است.
[MASK]: 'معادل'  : 0.05565404146909714
[MASK]: '،'  : 0.048520587384700775
[MASK]: 'برابر'  : 0.04669533669948578
معادل اسپانیا [MASK] است. 
[MASK]: 'فرانسه'  : 0.048739295452833176
[MASK]: 'اسپانیا'  : 0.0344146303832531
[MASK]: 'ایران'  : 0.02516143023967743
سوئد سوئدی فنلاند فنلاندی
سوئدی [MASK] سوئد است.
[MASK]: 'زبان'  : 0.1405027061700821
[MASK]: 'پایتخت'  : 0.04768168553709984
[MASK]: 'کشور'  : 0.04624209553003311
زبان فنلاند [MASK] است. 
[MASK]: 'فنلاندی'  : 0.26932621002197266
[MASK]: 'استونیایی'  : 0.12067663669586182
[MASK]: 'دانمارکی'  : 0.09128090739250183
بحرین بحرینی عراق عراقی
بحرینی [MASK] بحرین است.
[MASK]: 'متولد'  : 0.2512650787830353
[MASK]: 'کشور'  : 0.09095854312181473
[MASK]: 'شهروند'  : 0.07255789637565613
متولد عراق [MASK] است. 
[MASK]: 'بوده'  : 0.06961502879858017
[MASK]: 'شده'  : 0.05251045897603035
[MASK]: 'عراق'  : 0.04999599605798721
کویت کویتی سوریه سوری
کویتی [MASK] کویت است.
[MASK]: 'متولد'  : 0.

In [50]:
Accuracy= 0
y= df[df["cat"]=="gram_plural"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_plural"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x)
    z = ""+ df["third"][i[0]] + " " + predicted  +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

تالیف تالیفات مخلوق مخلوقات
تالیفات [MASK] تالیف است.
[MASK]: 'وی'  : 0.22904063761234283
[MASK]: 'او'  : 0.15257658064365387
[MASK]: 'ایشان'  : 0.1319533884525299
مخلوق وی[MASK] است. 
[MASK]: 'انسان'  : 0.125942200422287
[MASK]: 'ابلیس'  : 0.026886826381087303
[MASK]: 'ونوس'  : 0.02065483294427395
اثر اثرات ارزش ارزشها
اثرات [MASK] اثر است.
[MASK]: 'این'  : 0.0257883258163929
[MASK]: 'دو'  : 0.025220999494194984
[MASK]: 'زیر'  : 0.023319639265537262
ارزش این[MASK] است. 
[MASK]: 'کار'  : 0.01724035106599331
[MASK]: 'کتاب'  : 0.01544488687068224
[MASK]: 'پول'  : 0.011598627083003521
انقلابی انقلابیون مورد موارد
انقلابیون [MASK] انقلابی است.
[MASK]: '،'  : 0.06687462329864502
[MASK]: 'ایران'  : 0.04885024577379227
[MASK]: 'هر'  : 0.048106465488672256
مورد ،[MASK] است. 
[MASK]: 'مهم'  : 0.013618957251310349
[MASK]: 'ممکن'  : 0.010250142775475979
[MASK]: 'مسيله'  : 0.007996868342161179
حواری حواریون مورد موارد
حواریون [MASK] حواری است.
[MASK]: 'پطرس'  : 0.3310578763484955
[MASK]: 'عیسی'  :

Training

In [ ]:
!git clone https://github.com/dml-qom/FarsTail.git


Cloning into 'FarsTail'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 125 (delta 19), reused 18 (delta 18), pack-reused 104
Receiving objects: 100% (125/125), 43.86 MiB | 15.94 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_data = pd.read_csv('/content/FarsTail/data/Train-word.csv', sep='\t')
val_data = pd.read_csv('/content/FarsTail/data/Val-word.csv', sep='\t')
test_data = pd.read_csv('/content/FarsTail/data/Test-word.csv', sep='\t')

In [ ]:
!pip install transformers
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')
model = BertModel.from_pretrained('HooshvareLab/bert-fa-base-uncased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.3 MB/s eta 0:00:00


Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
label_encoder = {'e':0,'c':1,'n':2}
y_train = [label_encoder[i] for i in train_data['label'].values.tolist()]
y_valid = [label_encoder[i] for i in val_data['label'].values.tolist()]
y_test = [label_encoder[i] for i in test_data['label'].values.tolist()]

In [ ]:
def Tokenizer_persian(data):
  return tokenizer(data, padding=True, truncation=True, max_length=32)

train_p = Tokenizer_persian(train_data['premise'].values.tolist())
train_h = Tokenizer_persian(train_data['hypothesis'].values.tolist())

valid_p = Tokenizer_persian(val_data['premise'].values.tolist())
valid_h = Tokenizer_persian(val_data['hypothesis'].values.tolist())

test_p = Tokenizer_persian(test_data['premise'].values.tolist())
test_h = Tokenizer_persian(test_data['hypothesis'].values.tolist())


In [ ]:
train = {'input_ids':[], 'token_type_ids':[], 'attention_mask':[]}
valid = {'input_ids':[], 'token_type_ids':[], 'attention_mask':[]}
test = {'input_ids':[], 'token_type_ids':[], 'attention_mask':[]}

for i in range(len(train_p['input_ids'])):
  train['input_ids'].append(train_p['input_ids'][i]+train_h['input_ids'][i])
  train['token_type_ids'].append(train_p['token_type_ids'][i]+train_h['token_type_ids'][i])
  train['attention_mask'].append(train_p['attention_mask'][i]+train_h['attention_mask'][i])

for i in range(len(valid_p['input_ids'])):
  valid['input_ids'].append(valid_h['input_ids'][i]+valid_p['input_ids'][i])
  valid['token_type_ids'].append(valid_h['token_type_ids'][i]+valid_p['token_type_ids'][i])
  valid['attention_mask'].append(valid_h['attention_mask'][i]+valid_p['attention_mask'][i])


for i in range(len(test_h['input_ids'])):
  test['input_ids'].append(test_p['input_ids'][i]+test_h['input_ids'][i])
  test['token_type_ids'].append(test_p['token_type_ids'][i]+test_h['token_type_ids'][i])
  test['attention_mask'].append(test_p['attention_mask'][i]+test_h['attention_mask'][i])


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
model_name = "HooshvareLab/bert-fa-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

In [ ]:
train.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(train, y_train)
valid_dataset = Dataset(valid, y_valid)
test_dataset = Dataset(test)

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  636628 KB |  636628 KB |  636628 KB |       0 B  |\n|       from large pool |  636128 KB |  636128 KB |  636128 KB |       0 B  |\n|       from small pool |     500 KB |     500 KB |     500 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |  636628 KB |  636628 KB |  636628 KB |       0 B  |\n|       from large pool |  636128 KB |  636128 KB |

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7266
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2275
  Number of trainable parameters = 162843651


Step,Training Loss,Validation Loss
500,0.302300,3.759058
1000,0.239000,3.798706
1500,0.138600,4.619583
2000,0.059900,4.748284


***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-2000
Configuration saved in output/checkpoint-2000/config.json
Model weights saved in output/checkpoint-2000/pytorch_model.bin


Training completed. Do not fo

TrainOutput(global_step=2000, training_loss=0.18495973777770996, metrics={'train_runtime': 493.003, 'train_samples_per_second': 73.691, 'train_steps_per_second': 4.615, 'total_flos': 1050611876987904.0, 'train_loss': 0.18495973777770996, 'epoch': 4.4})

In [ ]:
trained_bert = BertForMaskedLM.from_pretrained('HooshvareLab/bert-fa-base-uncased')
tokenizer_trained_bert = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')

trained_bert.bert.encoder.layer=model.bert.encoder.layer

In [ ]:
Accuracy_city = 0
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "city"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "شهر "+df["first"][i[0]] + " در استان " + df["second"][i[0]] + " است در نتیجه شهر " + df["third"][i[0]] + " در استان " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_city = Accuracy_city + 1   
print(Accuracy_city)        



اهواز خوزستان قشم هرمزگان
شهر اهواز در استان خوزستان است در نتیجه شهر قشم در استان [MASK] است.


RuntimeError: ignored